<a href="https://colab.research.google.com/github/EdithOroche/IA/blob/main/Laboratorio05_SS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LABORATORIO 05


Crear una función prng_algoritmo_congruencial_cuadratico(X0,b,a,c,m), donde b es el coeficiente de X**2, a es el coeficiente de X y c es la constante aditiva, que devolverá una secuencia de números aleatorios todos diferentes.

Llamando a esta función, programar el prng_algoritmo_congruencial_lineal(b=0), prng_algoritmo_congruencial_multiplicativo(b=0, c=0) y  prng_algoritmo_congruencial_blum(b=1, a=0, c=0).

Luego, programe los algoritmos con tamaño de periodo condicionadoprng_algoritmo_congruencial_lineal_banks(X0,g,k,c) en donde se verifiquen las condiciones y luego se llame a prng_algoritmo_congruencial_lineal, prng_algoritmo_congruencial_multiplicativo_banks(X0,g,k) en donde se verifiquen las condiciones y luego se llame a prng_algoritmo_congruencial_multiplicativo, prng_algoritmo_congruencial_cuadratico_knuth(X0,g,b,a,c) en donde se verifiquen las condiciones y luego se llame a prng_algoritmo_congruencial_cuadratico. En caso de que no se verifiquen las condiciones, genere una excepción (raise exception).

1. Genere un periodo de N=128 utilizando los valores más pequeños posibles para b,c,a (en ese orden) que cumplan las condiciones que aseguren el periodo para el algoritmo congruencial lineal, multiplicativo y cuadrático. Indique los valores de b,c,a para cada uno.
2. Descubra el periodo del algoritmo de Blum con los valores p=3263052707, q=5847777359, X0=1033830034. Al verificar la condición de Blum, emplee el algoritmo de euclides (Parte I de: https://mate.dm.uba.ar/~pdenapo/apuntes-algebraI/2020/2do-cuatrimestre/clase-13-Euclides.pdf)

In [87]:
import math

**ALGORITMO CONGRUENCIAL CUADRATICO**

In [88]:
def cuadratico(b, a, c, x0):
    return (b * x0**2 + a * x0 + c)

def prng_algoritmo_congruencial_cuadratico(x0, b, a, c, m):
    r = []
    siguiente_semilla = x0
    while (True):
        siguiente_semilla = cuadratico(b, a, c, siguiente_semilla) % m
        nuevo_aleatorio = siguiente_semilla / (m - 1)
        if nuevo_aleatorio in r:
            break
        r.append(nuevo_aleatorio)
    return r

**ALGORITMO CONGRUENCIAL LINEAL**

In [89]:
def prng_algoritmo_congruencial_lineal (x0, a, c, m):
    r = prng_algoritmo_congruencial_cuadratico (x0, 0, a, c, m)
    return r

**ALGORITMO CONGRUENCIAL MULTIPLICATIVO**

In [90]:
def prng_algoritmo_congruencial_multiplicativo (x0, a, m):
    r=prng_algoritmo_congruencial_cuadratico (x0, 0, a, 0, m)
    return r

**ALGORITMO CONGRUENCIAL BLUM BLUM**

In [91]:
def MCD(a, b):
    if b == 0:
        return a
    else:
        return MCD(b, a % b)

def verifica(p, q):
    if p % 4 != 3 or q % 4 != 3:
        return False
    if MCD((p - 1) // 2, (q - 1) // 2) != 2:
        return False
    return True

def prng_algoritmo_congruencial_blum(x0, p, q, limite=1000):
    # Verificar condiciones de Blum
    if p % 4 != 3 or q % 4 != 3:
        raise Exception("p y q deben ser primos congruentes a 3 (mod 4).")

    m = p * q

    if math.gcd(x0, m) != 1:
        raise Exception("x0 no es coprimo con m.")

    r = []
    xi = x0 % m
    for _ in range(limite):
        xi = (xi * xi) % m
        ri = xi / (m - 1)
        if ri in r:
            break
        r.append(ri)
    return r


**ALGORITMO CONGRUENCIAL LINEAL BANKS**

In [92]:
def entero(g):
    return (isinstance (g, int)) or (isinstance(g, float) and g.is_integer())

def prng_algoritmo_congruencial_lineal_banks(x0, g, k, c):
    if (entero(g) and entero(k)):
        m = 2**g
        a = 1 + 4 * k

        if (MCD(c, m)==1):
            r = prng_algoritmo_congruencial_lineal(x0, a, c, m)
            return r
        else:
            raise Exception("c y m no son relativamente primos")
    else:
        raise Exception("g y k no son enteros")

**ALGORITMO CONGRUENCIAL MULTIPLICATIVO BANKS**

In [93]:
def impar (num):
    if(num%2== 1):
        return True
    else:
        return False

def prng_algoritmo_congruencial_multiplicativo_banks(x0, g, k):
    if(entero(k) and k > 0 and impar(x0)):
        m = 2**g
        a = 3 + 8 * k
        r = prng_algoritmo_congruencial_multiplicativo(x0, a, m)
        return r
    else:
        raise Exception("No es entero ni positivo")


**ALGORITMO CONGRUELCIAL CUADRATICO KNUTH**

In [94]:
def prng_algoritmo_congruencial_cuadratico_knuth(x0, g, b, a, c):

    if (impar(c) and impar(b)==False):
        if ((a - b) % 4 == 1):
            m = 2**g
            if g >= 3: # Condition 4
                r = prng_algoritmo_congruencial_cuadratico(x0, b, a, c, m)
                return r
            else:
                raise Exception("a - b mod 4 no es igul a 1")
        else:
            raise Exception("b no es par, o c es impar")

**ALGORITMO CONGRUENCIAL LINEAL - PERIODO**

In [106]:
def prng_algoritmo_congruencial_lineal_banks_periodo(x0, periodo):
    if periodo & (periodo - 1) != 0:
        raise Exception("El periodo debe ser una potencia de 2")
    g = int(math.log(periodo, 2))
    m = 2**g
    b = 0
    c = 1
    while MCD(c, m) != 1:
        c = c + 1
    k = 0
    a = 1
    valores = prng_algoritmo_congruencial_lineal_banks(x0, g, k, c)
    return b, c, a, valores

In [105]:
def multiplicativo(N):
    b = 0
    c = 0
    a = 11
    return b, c, a
def prng_algoritmo_congruencial_multiplicativo_banks_periodo(semilla, periodo):
    if periodo & (periodo - 1) != 0:
        raise Exception("El periodo debe ser una potencia de 2")
    if not impar(semilla):
        raise Exception("La semilla debe ser impar")

    m = periodo * 4  # recordar que N = m / 4
    g = int(math.log(m, 2))
    k = 1  # entero y positivo
    b, c, a = multiplicativo(periodo)
    valores = prng_algoritmo_congruencial_multiplicativo_banks(semilla, g, k)

    return b, c, a, valores


In [103]:
def cuadratico(N):
    for b in range(0, N):
        if b % 2 == 0:  # b par
            for c in range(1, N):
                if c % 2 == 1:  # c impar
                    for a in range(0, N):
                        if (a - b) % 4 == 1:
                            return b, c, a
    return None

def prng_algoritmo_congruencial_cuadratico_periodo(semilla, periodo):
    if periodo & (periodo - 1) != 0:
        raise Exception("El periodo debe ser una potencia de 2.")

    g = int(math.log(periodo, 2))
    b, c, a = cuadratico(periodo)
    valores = prng_algoritmo_congruencial_cuadratico_knuth(semilla, g, b, a, c)
    return b, c, a, valores

In [104]:
p = 3263652767
q = 5847777359
x0 = 10338388834

resultado = prng_algoritmo_congruencial_blum(x0, p, q, limite=500)

print("Cantidad generada:", len(resultado))
print("Valores generados:")

for r in resultado:
    print(r)

Cantidad generada: 500
Valores generados:
0.600295572515312
0.8462891645470816
0.9751981583875593
0.12880864086846683
0.8988036280590568
0.37323714447597406
0.9426631332072349
0.8717362457875347
0.9292118402726669
0.5856899386898987
0.5081088574606791
0.561437276779377
0.2534940454737507
0.05528721760170545
0.018102303689546073
0.16354077750281845
0.48200187466483435
0.9254889335452615
0.772940957952247
0.7701563591059861
0.751113621463251
0.010503505998733636
0.14529259186125065
0.49341188513968
0.08923872522004044
0.3752746191484153
0.033396858464617034
0.07477427496373759
0.6900771614663742
0.6461703592318464
0.5565676718298979
0.03974127865329062
0.8825191836749878
0.18181880055234234
0.3545768725444953
0.7159861408962961
0.6176432478603098
0.8803771428731816
0.6264495861218641
0.34358223262283416
0.8250544436289596
0.4972075757516844
0.34177545519925684
0.5309754899091979
0.3381208457437064
0.7699229633562695
0.5704335136277422
0.4624425364859176
0.4813611670998543
0.3804264054278